# Outline of thought process
- starting out with the creature class as the base class. The creature class will be used to create monsters/creatures that<br>
have not become npcs

- the pc_class class will be used to give player characters access to class mechanics
- the pc class will inherit from the creature class and the pc_class class to create a pc
- the npc class will inherit from the creature class and set the is_npc flag of an object to True so that a monster can be <br>
changed to npc status and stored to come back as a recurring character
- the item class will be used to create all items(including interactable objects our pcs might run into or use to change the <br> course of a battle. Think setting a fire to obscure the battle field or wedging a door shut


In [1]:
import pc_classes
import creatures
import races
import copy
import numpy as np
import random

from dice import Dice, DiceBag as DB, dice_avg
from skills import skills
from weapons import weapons as wpn
from armor import armor as ar

In [2]:
db = DB()
d = Dice()

In [3]:
### start of a creature class

class Creature():
    def __init__(self, creature, name = False):
        creature_attrs = creatures.creature_keys
        self.entity_type = 'monster'
        if name == False:
            self.name= creature['race']
        else:
            self.name= name
        for key in creature.keys():
            setattr(self, key, creature[key])
        for attr in creature_attrs:
            if hasattr(self, attr)==True:
                pass
            else:
                setattr(self, attr, False)
        self.disposition = 'hostile'
        self.life_state = 'alive'
        if hasattr(self, 'proficiencies') == False:
            self.proficiencies = []
            self.proficiencies.append(('unarmed_strike', 'r_trait'))
        else:
            self.proficiencies.append(('unarmed_strike', 'r_trait'))
            
    def ability_mod(self, abl):
        return creatures.ability_mod_tbl[self.abilities[abl]]
    
    def __dual_weilder_check(self):
        dual_weilder = False
        for prof in self.proficiencies:
            if 'dual_weilder' in prof:
                dual_weilder = True
        return dual_weilder
    
    def __armor_disadvantage(self):
        if self.disadvantage == False:
            self.disadvantage = []
            
        self.disadvantage.append('dex')
        self.disadvantage.append('str')
        self.disadvantage.append('attack')
        self.spell_casting = False
        for skill in self.skills:
            if self.skills[skill]['ability']== 'str' or self.skills[skill]['ability']== 'dex':
                self.disadvantage.append(skill)
                
    def __is_finesse_wpn(self, wep):
        return 'finesse' in wpn[wep]['properties']
    
    def __has_prof_bon(self):
        if hasattr(self, 'proficiency_bonus'):
            return self.proficiency_bonus
        else:
            return 0
        
    # get monster attack bonus. not used for pcs
    def __has_attack_bon(self):
        if hasattr(self,'attack_bonus'):
            return self.attack_bonus
        else:
            return False
                
    def equip_armor(self, armor):
        ac_base = armor['armor_class']['base']
        ac_dex_bon = armor['armor_class']['dex_mod']
        ac_dex_max = armor['armor_class']['dex_mod_max']
        ac_str_req = armor['str_req']
        ac_stealth = armor['stealth_penalty']
        
        if self.armor == False:
            self.armor = armor['name']
            if ac_str_req != False:
                self.ac = ac_base
                if self.abilities['str'] < ac_str_req:
                    self.movement -= 10
            if ac_dex_bon == True:
                if ac_dex_max != False:
                    self.ac = ac_base + ac_dex_max
                else:
                    self.ac = ac_base + self.ability_mod('dex')
            else:
                self.ac = ac_base
            
            for prof in self.proficiencies:
                if armor['type'] in prof:
                    pass
            else:
                self.__armor_disadvantage()

        else:
            print(self.armor + ' is already equiped.')
    
    def equip_shield(self, shield):
        if self.off_hand == False:
            self.ac += 2
            self.off_hand == shield['name']
            for prof in self.proficiencies:
                if shield['type'] in prof:
                    pass
            else:
                self.__armor_disadvantage()
        else:
            print(self.off_hand + ' is already equiped.')
            
    
    def equip_weapon(self, weapon, main_hand = True, off_hand = False, 
                     versatile=False,):
        
        dual_weilder = self.__dual_weilder_check()

        if off_hand == False and versatile == False:
            if self.main_hand != False:
                print(self.main_hand + ' is already equiped')
            else:
                if 'two_handed' in weapon['properties']:
                    self.main_hand = weapon['name']
                    self.off_hand = weapon['name']
                else:   
                    self.main_hand = weapon['name']
        if versatile == True:
            if self.off_hand != False:
                print(self.off_hand + 
                      ' is equipped in off_hand. Please unequip to use this weapons versatile properties')
            elif self.main_hand != False and self.main_hand != weapon['name']:
                print(self.main_hand +
                     ' is equipped. Please unequip this weapon to equip a new weapon')
            else:
                self.main_hand = weapon['name']
                self.off_hand = weapon['name']
                
        if off_hand == True:
            if self.off_hand != False:
                print(self.off_hand + ' is already equiped')
            elif self.main_hand != False:
                if'light' not in wpn[self.main_hand]['properties'] and dual_weilder == False:
                    print('You do not meet the requirements for dual weilding')
            elif 'light' not in weapon['proficiencies'] and dual_weilder == False:
                 print('You do not meet the requirements for dual weilding')
            else:
                self.off_hand = weapon['name']
        
    def unequip_weapon(self, main_hand = True, off_hand = False):
        if main_hand == True:
            self.main_hand = False
        if off_hand == True:
            self.off_hand = False   
    
    def damage_roll(self, wep):
        str_mod = self.ability_mod('str')
        dex_mod = self.ability_mod('dex')
        if self.main_hand != False and self.main_hand == self.off_hand:
            if 'versatile' in wpn[wep]['properties']:
                return -(d.roll_from_string(wpn[wep]['two_handed_damage']) + str_mod)
            else:
                return -(d.roll_from_string(wpn[wep]['damage']) + str_mod)
        elif 'finesse' in wpn[wep]['properties']:
            return -(d.roll_from_string(wpn[wep]['damage']) + max[str_mod, dex_mod])
        elif wpn[wep]['type'] == 'ranged':
            return -(d.roll_from_string(wpn[wep]['damage']) + dex_mod)
        else:
            return -(d.roll_from_string(wpn[wep]['damage']) + str_mod)


    #def un-equip_armor(self):
        #decriment ac
        #remove penilties if penilties exist
    
    #def add_inventory(self, itm):
       #add item to inventory
       #add weight to pc weight carried
       #apply penilties if encumbered
    
    #def cast(self, spell):
        #if spell.r_attack == True:
            #attack(spell)
        #elif spell.r_save == True:
        #else:
            #do a thing
            
            
    def attack_roll(self, wep, a_type = False):
        str_mod = self.ability_mod('str')
        dex_mod = self.ability_mod('dex')
        prof_bon = self.__has_prof_bon()
        
        if self.__has_attack_bon():
            return db.d20.roll() + self.attack_bonus
        else:
            if self.__is_finesse_wpn(wep):
                return db.d20.roll() + prof_bon + max[str_mod, dex_mod]
            elif a_type == 'spell':
                c_abil_mod = creatures.ability_mod_tbl[self.abilities[self.cast_ability]]
                return db.d20.roll() + prof_bon + c_abil_mod
            elif a_type == 'ranged':
                return db.d20.roll() + prof_bon + dex_mod
            else:
                return db.d20.roll() + prof_bon + str_mod
            
    def saving_throw(self, save_stat):
        prof = False
        abl_mod = self.ability_mod(save_stat)
        
        if hasattr(self, 'saving_throws') == False:
            return db.d20.roll() + abl_mod
        else:
            for prof in self.saving_throws:
                if save_stat in prof:
                    prof = True
            if prof== True:
                return db.d20.roll() + self.proficiency_bonus + abl_mod
            else:
                return db.d20.roll() + abl_mod
        
    #def skill_check(self, skill):
         #return db.d20.roll() + prof(ifpro) + abil_mod
    
    #def dash(self, movement):
        #return self.movement * 2
    
    #def disengage(self):
      #self.allow_aoo = False
    
    #def dodge(self.advantage_toggle):
        #attacker.disadvantage = True
    
    #def help(self,target):
        #ally.advantage = true
    
    #def hide(self, stealth_check):
        #check for cover
        #skill_check('stealth')
    
    #def ready(self, trigger, action):
    
    #def search(self, perception_check):
    
    #def use_an_object(self):
    
    #def imporovise(self, skill_check):

In [4]:
class Pc(Creature):
    def __init__(self):
        pass
    def __get_pc_lvl(self, creature):
        lvl = 0
        for pc_class in creature.pc_classes:
            lvl += creature.pc_classes[pc_class]
        return lvl
    def create_pc(self, creature):
        creature.entity_type = 'pc'
        creature.disposition = 'ally'
        creature.pc =  True
        creature.pc_lvl = self.__get_pc_lvl(creature)
        creature.proficiency_bonus = pc_classes.prof_bonus[creature.pc_lvl]
        creature.skills = copy.deepcopy(skills)
        creature.initiative_bonus = creatures.ability_mod_tbl[creature.abilities['dex']]
        if creature.armor == False and creature.ac == False:
            dex_mod = creatures.ability_mod_tbl[creature.abilities['dex']]
            wis_mod = creatures.ability_mod_tbl[creature.abilities['wis']]
            con_mod = creatures.ability_mod_tbl[creature.abilities['con']]
            barb_ua = 10 + con_mod + dex_mod
            monk_ua = 10 + dex_mod + wis_mod
            if 'barbarian' in creature.pc_classes.keys() & 'monk' in creature.pc_classes.keys():
                unarmored_defense = []
                unarmored_defense.append(barb_ua)
                unarmored_defense.append(monk_ua)
                creature.ac = max(unarmored_defense)
            elif 'barbarian' in creature.pc_classes.keys():
                creature.ac = barb_ua
            elif 'monk' in creature.pc_classes.keys():
                creature.ac = monk_ua
            else:
                creature.ac = 10 + creatures.ability_mod_tbl[creature.abilities['dex']]
        

In [5]:
# keeping pc_class and pc as different classes to account for multi-classing

class PcClass:
    def __init__(self):
        pass
    
    def __hill_dwarf_hp_bonus(self, creature):
        creature.hp_max += 1
        
    def add_first_class(self, creature, pc_class, gen_method):
        p_class = pc_class['pc_class']
        creature.pc_classes = {p_class:1}
        creature.abilities =  gen_method
        
        ### adds first class attrs to creature
        for key in pc_class.keys():
            if key=='pc_class':
                pass
            else:
                setattr(creature, key, pc_class[key])
                
        ### adds racial bonuses to ability scores 
        for key in creature.ability_score_increase.keys():
            creature.abilities[key] += creature.ability_score_increase[key]
        
        #Turns creature with class into pc
        pc = Pc()
        pc.create_pc(creature)
        
        #move class profs into creature profs
        for prof in creature.arm_weap_prof:
            creature.proficiencies.append(prof)
            
        
        ### sets hp_max
        con_bonus = creatures.ability_mod_tbl[creature.abilities['con']]
        hp_max = int(pc_class['hit_die'][1:]) + con_bonus
        creature.hp_max = hp_max
        if creature.subrace == 'hill':
            self.__hill_dwarf_hp_bonus(creature)
        creature.hp = hp_max
            
    def add_class_lvl(self, creature, p_class_name):
        ### update hp_max
        class_hp_add = dice_avg[creature.hit_die]
        con_bonus = creatures.ability_mod_tbl[creature.abilities['con']]
        creature.hp_max += class_hp_add + con_bonus
        if creature.subrace == 'hill':
            self.__hill_dwarf_hp_bonus(creature)
        p_class_exists = False
        for p_class in creature.pc_classes:
            if p_class == p_class_name:
                p_class_exists = True
        if p_class_exists == False:
            creature.pc_classes[p_class_name] = 1
            creature.pc_lvl += 1
            creature.proficiency_bonus = pc_classes.prof_bonus[creature.pc_lvl]
        else:
            creature.pc_classes[p_class_name] += 1
            creature.pc_lvl += 1
            creature.proficiency_bonus = pc_classes.prof_bonus[creature.pc_lvl]


In [6]:
class Npc():
    def __init__(self):
        pass
    def is_npc(self, creature):
        creature.entity_type = 'npc'
        creature.entity_type = 'indifferent'
        setattr(creature, 'npc', True)
        setattr(creature, 'max_hp', 0)

In [7]:
class item:
    def __init__(self):
        pass
    

In [8]:
class CombatMechanics:
    def __init__(self, combatants, e_combatants):
        self.round = 1
        for c in e_combatants:
            c.disposition = 'hostile'
        self.combatants = combatants + e_combatants
        #self.c_area = c_area

    def __order_combat(self):
        for i in range(1, len(self.combatants)):
            key_item = self.combatants[i]
            key_item_num = key_item.initiative
            
            j = i-1
            
            while j>=0 and self.combatants[j].initiative < key_item_num:
                self.combatants[j+1] = self.combatants[j]
                j -= 1
                
            self.combatants[j + 1] = key_item
        
    def initiative(self):
        for c in self.combatants:
            advantage = hasattr(c, 'advantage')
            disadvantage = hasattr(c, 'disadvantage')
            init_advantage = False
            init_disadvantage = True
            
            if advantage == True:
                if 'initiative' or 'dex' in c.advantage:
                    init_advantage = True
            if disadvantage == True:
                if 'initiative' or 'dex' in c.disadvantage:
                    init_disadvantage = True
            if init_advantage == True and init_disadvantage == True:
                setattr(c, 'initiative', db.d20.roll() + c.initiative_bonus + c.abilities['dex']/100)
            elif init_advantage == True:
                setattr(c, 'initiative', db.roll_advantage() + c.initiative_bonus + c.abilities['dex']/100)
            elif init_disadvantage == True:
                setattr(c, 'initiative', db.roll_disadvantage() + c.initiative_bonus + c.abilities['dex']/100)
            else:
                setattr(c, 'initiative', db.d20.roll() + c.initiative_bonus + c.abilities['dex']/100)
        self.__order_combat()
    def get_target(self, attacker):
        combatants = self.combatants
        for i, c in enumerate(combatants):
            if c.hp <= 0:
                if i == len(self.combatants):
                    return False
                else:
                    pass
            elif c.disposition == attacker.disposition:
                if i == len(self.combatants):
                    return False
                else:
                    pass
            else:
                return c
            
    def apply_hp(self, target, hp):
        target.hp = target.hp + hp
        
            
    def attack_action(self, attacker, strategy = False):
        target = self.get_target(attacker)
        if target == False or target is None:
            return target
        else:
            a_roll = attacker.attack_roll(attacker.main_hand)
            if a_roll >= target.ac:
                o_hp = target.hp
                dmg = attacker.damage_roll(attacker.main_hand)
                self.apply_hp(target, dmg)
                return {'name': attacker.name, 'target': target.name, 'a_roll':a_roll,
                       'damage': dmg}
            else:
                return {'name': attacker.name, 'target': target.name, 'a_roll':a_roll}
                
    def death_save(self, pc):
        r = db.d20.roll()
        if r == 1:
            pc.death_save_count['f'] += 2
            if pc.death_save_count['f'] >= 3:
                pc.life_state = 'dead'
        elif r == 20:
            pc.death_save_count['s'] += 2
            if pc.death_save_count['s']>= 3:
                pc.life_state = 'stable'
        elif r < 10:
            pc.death_save_count['f'] += 1
            if pc.death_save_count['f'] >= 3:
                pc.life_state = 'dead'
        else:
            pc.death_save_count['s'] += 1
            if pc.death_save_count['s']>= 3:
                pc.life_state = 'stable'            

    def dying(self, pc):
        if pc.disposition == 'ally':
            if pc.life_state == 'alive':
                pc.life_state = 'dying'
                setattr(pc, 'death_save_count', {'s': 0, 'f': 0})
                self.death_save(pc)
            elif pc.life_state == 'dying':
                self.death_save(pc)
                
    def combat(self):
        rnd = 1
        alive = True
        winner = 0
        while alive==True:
            for c in self.combatants:
                hp = c.hp
                c_disposition = c.disposition 
                if hp <= 0:
                    if c_disposition == 'ally':
                        self.dying(c)
                    else:
                        c.life_state = 'dead'
                        pass
                else:
                    a = self.attack_action(c)
                    if a==False:
                        alive = False
                    elif a is None:
                        alive = False
            rnd += 1
        pc_death = [c.life_state for c in self.combatants if c.disposition == 'ally']
        if 'alive' in pc_death:
            winner = 1
        else:
            winner = -1
        dead_count = len([state for state in pc_death if state == 'dead'])
        return winner, dead_count, pc_death
    
    def test_combat(self, num_tests):
        results = []
        death_counts = []
        death_details = []

        for _ in range(num_tests):
            gc = copy.deepcopy([c for c in self.combatants if c.disposition == 'ally'])
            ec = copy.deepcopy([c for c in self.combatants if c.disposition == 'hostile'])
            c_bat = CombatMechanics(gc, ec)
            c_bat.initiative()
            c = c_bat.combat()
            results.append(c[0])
            death_counts.append(c[1])
            death_details.append(c[2])
            
        difficulty = sum(results)/len(results)
        return results, death_counts, death_details, difficulty
    
        

In [9]:
tiny = Creature(races.dwarf_hill, 'tiny')
pc_class = PcClass()

In [10]:
tiny.abilities

False

In [11]:
pc_class.add_first_class(tiny, pc_classes.barbarian, pc_classes.barbarian['ability_by_method']['pb'])

In [12]:
for _ in range(14):
    pc_class.add_class_lvl(tiny, 'barbarian')
for _ in range(5):
    pc_class.add_class_lvl(tiny, 'fighter')

In [13]:
### consume spell slot command

In [14]:
tiny.equip_weapon(wpn['greataxe'])
tiny1 = copy.deepcopy(tiny)
tiny2 = copy.deepcopy(tiny)
tiny3 = copy.deepcopy(tiny)
evil_tiny = copy.deepcopy(tiny)
evil_tiny.disposition = 'hostile'
evil_tiny.name = 'evil_tiny'

In [15]:
#aar = Creature(creatures.aarakocra)
#aar.equip_weapon(wpn['javelin'], versatile = True)
#aar1 = copy.deepcopy(aar)
#aar2 = copy.deepcopy(aar)
#aar3 = copy.deepcopy(aar)
evil_tiny1 = copy.deepcopy(evil_tiny)
evil_tiny2 = copy.deepcopy(evil_tiny)
evil_tiny3 = copy.deepcopy(evil_tiny)

In [16]:
tiny.pc_lvl

20

In [17]:
g = [tiny, tiny1, tiny2, tiny3]
e = [evil_tiny, evil_tiny1, evil_tiny2, evil_tiny3]

In [18]:
cmbt = CombatMechanics(g, e)

In [19]:
#tests = cmbt.test_combat(10000)

In [20]:
#len(tests)

In [21]:
#tests[3]

In [22]:
#d = len([t for t in tests[1] if t>0])
#cd = d/len(tests[1])
#cd

# Playing with creating area
 - currently map is working like a table with the first thre columns being the x, y, z coordinates respectively and the rest of the columns containing information about the specific coordinate.
 - table headings of the matrix would be x, y, z, Terrain features, construction features, other objects, creature 

In [23]:
class AreaMap:
    def __init__(self):
        pass
    
    def create_map(self, tup: tuple):
        self.area_map = np.zeros(tup, dtype = object)
                    
    def get_reachable_area(self, coords: tuple, area, a_range):
        vals = []
        x, y, z = coords
        x_lim, y_lim, z_lim = area.shape
        low_lim = 0
        for x_off in range(-a_range, a_range):
            for y_off in range(-a_range, a_range):
                for z_off in range(-a_range, a_range):
                    if x_off == y_off == z_off == 0:
                        continue
                    elif (x+x_off)<0 or (y+y_off)<0 or (z+z_off)<0:
                        continue
                    elif (x+x_off)>=x_lim or (y+y_off)>=y_lim or (z+z_off)>=z_lim:
                        continue
                       
                vals.append((x + x_off, y + y_off, z + z_off))
        
        return vals                    

    def add_object(self, area, tup: tuple, obj_key):
        area[tup[0]][tup[1]][tup[2]] = obj_key
    
    #def add_creature(self, area, creatures):
    #    self.creatures = []
    #    pg = PINGenerator()
    #    for c in creatures:
    #        pin = pg('c')
    #        if pin in self.creatures:
                
    

In [24]:
 #def get_adjs(tup: tuple, area, a_range):
 #       vals = []
 #       x, y, z = tup
 #       x_lim, y_lim, z_lim = area
 #       low_lim = 0
 #       for x_off in range(-a_range, a_range):
 #           for y_off in range(-a_range, a_range):
 #               for z_off in range(-a_range, a_range):
 #                   if x_off == y_off == z_off == 0:
 #                       continue
 #                   elif (x+x_off)<0 or (y+y_off)<0 or (z+z_off)<0:
 #                       continue
 #                   elif (x+x_off)>=x_lim or (y+y_off)>=y_lim or (z+z_off)>=z_lim:
 #                       continue
 #                   
 #                   
 #                   vals.append((x + x_off, y + y_off, z + z_off))
 #       return vals

In [25]:
x = AreaMap()

In [26]:
x.create_map((20, 6, 20))

In [27]:
x.area_map

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ...,

In [28]:
x.area_map.shape

(20, 6, 20)

In [29]:
len(x.area_map)

20

In [44]:
for i in range(-11, 12):
    for j in range(-11, 12):
        for k in range(-11, 12):
            print(i, j, k)

-11 -11 -11
-11 -11 -10
-11 -11 -9
-11 -11 -8
-11 -11 -7
-11 -11 -6
-11 -11 -5
-11 -11 -4
-11 -11 -3
-11 -11 -2
-11 -11 -1
-11 -11 0
-11 -11 1
-11 -11 2
-11 -11 3
-11 -11 4
-11 -11 5
-11 -11 6
-11 -11 7
-11 -11 8
-11 -11 9
-11 -11 10
-11 -11 11
-11 -10 -11
-11 -10 -10
-11 -10 -9
-11 -10 -8
-11 -10 -7
-11 -10 -6
-11 -10 -5
-11 -10 -4
-11 -10 -3
-11 -10 -2
-11 -10 -1
-11 -10 0
-11 -10 1
-11 -10 2
-11 -10 3
-11 -10 4
-11 -10 5
-11 -10 6
-11 -10 7
-11 -10 8
-11 -10 9
-11 -10 10
-11 -10 11
-11 -9 -11
-11 -9 -10
-11 -9 -9
-11 -9 -8
-11 -9 -7
-11 -9 -6
-11 -9 -5
-11 -9 -4
-11 -9 -3
-11 -9 -2
-11 -9 -1
-11 -9 0
-11 -9 1
-11 -9 2
-11 -9 3
-11 -9 4
-11 -9 5
-11 -9 6
-11 -9 7
-11 -9 8
-11 -9 9
-11 -9 10
-11 -9 11
-11 -8 -11
-11 -8 -10
-11 -8 -9
-11 -8 -8
-11 -8 -7
-11 -8 -6
-11 -8 -5
-11 -8 -4
-11 -8 -3
-11 -8 -2
-11 -8 -1
-11 -8 0
-11 -8 1
-11 -8 2
-11 -8 3
-11 -8 4
-11 -8 5
-11 -8 6
-11 -8 7
-11 -8 8
-11 -8 9
-11 -8 10
-11 -8 11
-11 -7 -11
-11 -7 -10
-11 -7 -9
-11 -7 -8
-11 -7 -7
-11 -7 -6
-11 

-9 -10 -4
-9 -10 -3
-9 -10 -2
-9 -10 -1
-9 -10 0
-9 -10 1
-9 -10 2
-9 -10 3
-9 -10 4
-9 -10 5
-9 -10 6
-9 -10 7
-9 -10 8
-9 -10 9
-9 -10 10
-9 -10 11
-9 -9 -11
-9 -9 -10
-9 -9 -9
-9 -9 -8
-9 -9 -7
-9 -9 -6
-9 -9 -5
-9 -9 -4
-9 -9 -3
-9 -9 -2
-9 -9 -1
-9 -9 0
-9 -9 1
-9 -9 2
-9 -9 3
-9 -9 4
-9 -9 5
-9 -9 6
-9 -9 7
-9 -9 8
-9 -9 9
-9 -9 10
-9 -9 11
-9 -8 -11
-9 -8 -10
-9 -8 -9
-9 -8 -8
-9 -8 -7
-9 -8 -6
-9 -8 -5
-9 -8 -4
-9 -8 -3
-9 -8 -2
-9 -8 -1
-9 -8 0
-9 -8 1
-9 -8 2
-9 -8 3
-9 -8 4
-9 -8 5
-9 -8 6
-9 -8 7
-9 -8 8
-9 -8 9
-9 -8 10
-9 -8 11
-9 -7 -11
-9 -7 -10
-9 -7 -9
-9 -7 -8
-9 -7 -7
-9 -7 -6
-9 -7 -5
-9 -7 -4
-9 -7 -3
-9 -7 -2
-9 -7 -1
-9 -7 0
-9 -7 1
-9 -7 2
-9 -7 3
-9 -7 4
-9 -7 5
-9 -7 6
-9 -7 7
-9 -7 8
-9 -7 9
-9 -7 10
-9 -7 11
-9 -6 -11
-9 -6 -10
-9 -6 -9
-9 -6 -8
-9 -6 -7
-9 -6 -6
-9 -6 -5
-9 -6 -4
-9 -6 -3
-9 -6 -2
-9 -6 -1
-9 -6 0
-9 -6 1
-9 -6 2
-9 -6 3
-9 -6 4
-9 -6 5
-9 -6 6
-9 -6 7
-9 -6 8
-9 -6 9
-9 -6 10
-9 -6 11
-9 -5 -11
-9 -5 -10
-9 -5 -9
-9 -5 -8
-9 -5 -7
-9 -5 -

-7 5 3
-7 5 4
-7 5 5
-7 5 6
-7 5 7
-7 5 8
-7 5 9
-7 5 10
-7 5 11
-7 6 -11
-7 6 -10
-7 6 -9
-7 6 -8
-7 6 -7
-7 6 -6
-7 6 -5
-7 6 -4
-7 6 -3
-7 6 -2
-7 6 -1
-7 6 0
-7 6 1
-7 6 2
-7 6 3
-7 6 4
-7 6 5
-7 6 6
-7 6 7
-7 6 8
-7 6 9
-7 6 10
-7 6 11
-7 7 -11
-7 7 -10
-7 7 -9
-7 7 -8
-7 7 -7
-7 7 -6
-7 7 -5
-7 7 -4
-7 7 -3
-7 7 -2
-7 7 -1
-7 7 0
-7 7 1
-7 7 2
-7 7 3
-7 7 4
-7 7 5
-7 7 6
-7 7 7
-7 7 8
-7 7 9
-7 7 10
-7 7 11
-7 8 -11
-7 8 -10
-7 8 -9
-7 8 -8
-7 8 -7
-7 8 -6
-7 8 -5
-7 8 -4
-7 8 -3
-7 8 -2
-7 8 -1
-7 8 0
-7 8 1
-7 8 2
-7 8 3
-7 8 4
-7 8 5
-7 8 6
-7 8 7
-7 8 8
-7 8 9
-7 8 10
-7 8 11
-7 9 -11
-7 9 -10
-7 9 -9
-7 9 -8
-7 9 -7
-7 9 -6
-7 9 -5
-7 9 -4
-7 9 -3
-7 9 -2
-7 9 -1
-7 9 0
-7 9 1
-7 9 2
-7 9 3
-7 9 4
-7 9 5
-7 9 6
-7 9 7
-7 9 8
-7 9 9
-7 9 10
-7 9 11
-7 10 -11
-7 10 -10
-7 10 -9
-7 10 -8
-7 10 -7
-7 10 -6
-7 10 -5
-7 10 -4
-7 10 -3
-7 10 -2
-7 10 -1
-7 10 0
-7 10 1
-7 10 2
-7 10 3
-7 10 4
-7 10 5
-7 10 6
-7 10 7
-7 10 8
-7 10 9
-7 10 10
-7 10 11
-7 11 -11
-7 11 -10
-7 11 -9
-7 

-5 7 10
-5 7 11
-5 8 -11
-5 8 -10
-5 8 -9
-5 8 -8
-5 8 -7
-5 8 -6
-5 8 -5
-5 8 -4
-5 8 -3
-5 8 -2
-5 8 -1
-5 8 0
-5 8 1
-5 8 2
-5 8 3
-5 8 4
-5 8 5
-5 8 6
-5 8 7
-5 8 8
-5 8 9
-5 8 10
-5 8 11
-5 9 -11
-5 9 -10
-5 9 -9
-5 9 -8
-5 9 -7
-5 9 -6
-5 9 -5
-5 9 -4
-5 9 -3
-5 9 -2
-5 9 -1
-5 9 0
-5 9 1
-5 9 2
-5 9 3
-5 9 4
-5 9 5
-5 9 6
-5 9 7
-5 9 8
-5 9 9
-5 9 10
-5 9 11
-5 10 -11
-5 10 -10
-5 10 -9
-5 10 -8
-5 10 -7
-5 10 -6
-5 10 -5
-5 10 -4
-5 10 -3
-5 10 -2
-5 10 -1
-5 10 0
-5 10 1
-5 10 2
-5 10 3
-5 10 4
-5 10 5
-5 10 6
-5 10 7
-5 10 8
-5 10 9
-5 10 10
-5 10 11
-5 11 -11
-5 11 -10
-5 11 -9
-5 11 -8
-5 11 -7
-5 11 -6
-5 11 -5
-5 11 -4
-5 11 -3
-5 11 -2
-5 11 -1
-5 11 0
-5 11 1
-5 11 2
-5 11 3
-5 11 4
-5 11 5
-5 11 6
-5 11 7
-5 11 8
-5 11 9
-5 11 10
-5 11 11
-4 -11 -11
-4 -11 -10
-4 -11 -9
-4 -11 -8
-4 -11 -7
-4 -11 -6
-4 -11 -5
-4 -11 -4
-4 -11 -3
-4 -11 -2
-4 -11 -1
-4 -11 0
-4 -11 1
-4 -11 2
-4 -11 3
-4 -11 4
-4 -11 5
-4 -11 6
-4 -11 7
-4 -11 8
-4 -11 9
-4 -11 10
-4 -11 11
-4 -10 -11
-

-2 5 4
-2 5 5
-2 5 6
-2 5 7
-2 5 8
-2 5 9
-2 5 10
-2 5 11
-2 6 -11
-2 6 -10
-2 6 -9
-2 6 -8
-2 6 -7
-2 6 -6
-2 6 -5
-2 6 -4
-2 6 -3
-2 6 -2
-2 6 -1
-2 6 0
-2 6 1
-2 6 2
-2 6 3
-2 6 4
-2 6 5
-2 6 6
-2 6 7
-2 6 8
-2 6 9
-2 6 10
-2 6 11
-2 7 -11
-2 7 -10
-2 7 -9
-2 7 -8
-2 7 -7
-2 7 -6
-2 7 -5
-2 7 -4
-2 7 -3
-2 7 -2
-2 7 -1
-2 7 0
-2 7 1
-2 7 2
-2 7 3
-2 7 4
-2 7 5
-2 7 6
-2 7 7
-2 7 8
-2 7 9
-2 7 10
-2 7 11
-2 8 -11
-2 8 -10
-2 8 -9
-2 8 -8
-2 8 -7
-2 8 -6
-2 8 -5
-2 8 -4
-2 8 -3
-2 8 -2
-2 8 -1
-2 8 0
-2 8 1
-2 8 2
-2 8 3
-2 8 4
-2 8 5
-2 8 6
-2 8 7
-2 8 8
-2 8 9
-2 8 10
-2 8 11
-2 9 -11
-2 9 -10
-2 9 -9
-2 9 -8
-2 9 -7
-2 9 -6
-2 9 -5
-2 9 -4
-2 9 -3
-2 9 -2
-2 9 -1
-2 9 0
-2 9 1
-2 9 2
-2 9 3
-2 9 4
-2 9 5
-2 9 6
-2 9 7
-2 9 8
-2 9 9
-2 9 10
-2 9 11
-2 10 -11
-2 10 -10
-2 10 -9
-2 10 -8
-2 10 -7
-2 10 -6
-2 10 -5
-2 10 -4
-2 10 -3
-2 10 -2
-2 10 -1
-2 10 0
-2 10 1
-2 10 2
-2 10 3
-2 10 4
-2 10 5
-2 10 6
-2 10 7
-2 10 8
-2 10 9
-2 10 10
-2 10 11
-2 11 -11
-2 11 -10
-2 11 -9
-2 11 -8
-

1 8 -3
1 8 -2
1 8 -1
1 8 0
1 8 1
1 8 2
1 8 3
1 8 4
1 8 5
1 8 6
1 8 7
1 8 8
1 8 9
1 8 10
1 8 11
1 9 -11
1 9 -10
1 9 -9
1 9 -8
1 9 -7
1 9 -6
1 9 -5
1 9 -4
1 9 -3
1 9 -2
1 9 -1
1 9 0
1 9 1
1 9 2
1 9 3
1 9 4
1 9 5
1 9 6
1 9 7
1 9 8
1 9 9
1 9 10
1 9 11
1 10 -11
1 10 -10
1 10 -9
1 10 -8
1 10 -7
1 10 -6
1 10 -5
1 10 -4
1 10 -3
1 10 -2
1 10 -1
1 10 0
1 10 1
1 10 2
1 10 3
1 10 4
1 10 5
1 10 6
1 10 7
1 10 8
1 10 9
1 10 10
1 10 11
1 11 -11
1 11 -10
1 11 -9
1 11 -8
1 11 -7
1 11 -6
1 11 -5
1 11 -4
1 11 -3
1 11 -2
1 11 -1
1 11 0
1 11 1
1 11 2
1 11 3
1 11 4
1 11 5
1 11 6
1 11 7
1 11 8
1 11 9
1 11 10
1 11 11
2 -11 -11
2 -11 -10
2 -11 -9
2 -11 -8
2 -11 -7
2 -11 -6
2 -11 -5
2 -11 -4
2 -11 -3
2 -11 -2
2 -11 -1
2 -11 0
2 -11 1
2 -11 2
2 -11 3
2 -11 4
2 -11 5
2 -11 6
2 -11 7
2 -11 8
2 -11 9
2 -11 10
2 -11 11
2 -10 -11
2 -10 -10
2 -10 -9
2 -10 -8
2 -10 -7
2 -10 -6
2 -10 -5
2 -10 -4
2 -10 -3
2 -10 -2
2 -10 -1
2 -10 0
2 -10 1
2 -10 2
2 -10 3
2 -10 4
2 -10 5
2 -10 6
2 -10 7
2 -10 8
2 -10 9
2 -10 10
2 -10 11
2 

4 6 -8
4 6 -7
4 6 -6
4 6 -5
4 6 -4
4 6 -3
4 6 -2
4 6 -1
4 6 0
4 6 1
4 6 2
4 6 3
4 6 4
4 6 5
4 6 6
4 6 7
4 6 8
4 6 9
4 6 10
4 6 11
4 7 -11
4 7 -10
4 7 -9
4 7 -8
4 7 -7
4 7 -6
4 7 -5
4 7 -4
4 7 -3
4 7 -2
4 7 -1
4 7 0
4 7 1
4 7 2
4 7 3
4 7 4
4 7 5
4 7 6
4 7 7
4 7 8
4 7 9
4 7 10
4 7 11
4 8 -11
4 8 -10
4 8 -9
4 8 -8
4 8 -7
4 8 -6
4 8 -5
4 8 -4
4 8 -3
4 8 -2
4 8 -1
4 8 0
4 8 1
4 8 2
4 8 3
4 8 4
4 8 5
4 8 6
4 8 7
4 8 8
4 8 9
4 8 10
4 8 11
4 9 -11
4 9 -10
4 9 -9
4 9 -8
4 9 -7
4 9 -6
4 9 -5
4 9 -4
4 9 -3
4 9 -2
4 9 -1
4 9 0
4 9 1
4 9 2
4 9 3
4 9 4
4 9 5
4 9 6
4 9 7
4 9 8
4 9 9
4 9 10
4 9 11
4 10 -11
4 10 -10
4 10 -9
4 10 -8
4 10 -7
4 10 -6
4 10 -5
4 10 -4
4 10 -3
4 10 -2
4 10 -1
4 10 0
4 10 1
4 10 2
4 10 3
4 10 4
4 10 5
4 10 6
4 10 7
4 10 8
4 10 9
4 10 10
4 10 11
4 11 -11
4 11 -10
4 11 -9
4 11 -8
4 11 -7
4 11 -6
4 11 -5
4 11 -4
4 11 -3
4 11 -2
4 11 -1
4 11 0
4 11 1
4 11 2
4 11 3
4 11 4
4 11 5
4 11 6
4 11 7
4 11 8
4 11 9
4 11 10
4 11 11
5 -11 -11
5 -11 -10
5 -11 -9
5 -11 -8
5 -11 -7
5 -11 -6
5 -

7 -6 -2
7 -6 -1
7 -6 0
7 -6 1
7 -6 2
7 -6 3
7 -6 4
7 -6 5
7 -6 6
7 -6 7
7 -6 8
7 -6 9
7 -6 10
7 -6 11
7 -5 -11
7 -5 -10
7 -5 -9
7 -5 -8
7 -5 -7
7 -5 -6
7 -5 -5
7 -5 -4
7 -5 -3
7 -5 -2
7 -5 -1
7 -5 0
7 -5 1
7 -5 2
7 -5 3
7 -5 4
7 -5 5
7 -5 6
7 -5 7
7 -5 8
7 -5 9
7 -5 10
7 -5 11
7 -4 -11
7 -4 -10
7 -4 -9
7 -4 -8
7 -4 -7
7 -4 -6
7 -4 -5
7 -4 -4
7 -4 -3
7 -4 -2
7 -4 -1
7 -4 0
7 -4 1
7 -4 2
7 -4 3
7 -4 4
7 -4 5
7 -4 6
7 -4 7
7 -4 8
7 -4 9
7 -4 10
7 -4 11
7 -3 -11
7 -3 -10
7 -3 -9
7 -3 -8
7 -3 -7
7 -3 -6
7 -3 -5
7 -3 -4
7 -3 -3
7 -3 -2
7 -3 -1
7 -3 0
7 -3 1
7 -3 2
7 -3 3
7 -3 4
7 -3 5
7 -3 6
7 -3 7
7 -3 8
7 -3 9
7 -3 10
7 -3 11
7 -2 -11
7 -2 -10
7 -2 -9
7 -2 -8
7 -2 -7
7 -2 -6
7 -2 -5
7 -2 -4
7 -2 -3
7 -2 -2
7 -2 -1
7 -2 0
7 -2 1
7 -2 2
7 -2 3
7 -2 4
7 -2 5
7 -2 6
7 -2 7
7 -2 8
7 -2 9
7 -2 10
7 -2 11
7 -1 -11
7 -1 -10
7 -1 -9
7 -1 -8
7 -1 -7
7 -1 -6
7 -1 -5
7 -1 -4
7 -1 -3
7 -1 -2
7 -1 -1
7 -1 0
7 -1 1
7 -1 2
7 -1 3
7 -1 4
7 -1 5
7 -1 6
7 -1 7
7 -1 8
7 -1 9
7 -1 10
7 -1 11
7 0 -11
7 0 -10
7 

9 5 -10
9 5 -9
9 5 -8
9 5 -7
9 5 -6
9 5 -5
9 5 -4
9 5 -3
9 5 -2
9 5 -1
9 5 0
9 5 1
9 5 2
9 5 3
9 5 4
9 5 5
9 5 6
9 5 7
9 5 8
9 5 9
9 5 10
9 5 11
9 6 -11
9 6 -10
9 6 -9
9 6 -8
9 6 -7
9 6 -6
9 6 -5
9 6 -4
9 6 -3
9 6 -2
9 6 -1
9 6 0
9 6 1
9 6 2
9 6 3
9 6 4
9 6 5
9 6 6
9 6 7
9 6 8
9 6 9
9 6 10
9 6 11
9 7 -11
9 7 -10
9 7 -9
9 7 -8
9 7 -7
9 7 -6
9 7 -5
9 7 -4
9 7 -3
9 7 -2
9 7 -1
9 7 0
9 7 1
9 7 2
9 7 3
9 7 4
9 7 5
9 7 6
9 7 7
9 7 8
9 7 9
9 7 10
9 7 11
9 8 -11
9 8 -10
9 8 -9
9 8 -8
9 8 -7
9 8 -6
9 8 -5
9 8 -4
9 8 -3
9 8 -2
9 8 -1
9 8 0
9 8 1
9 8 2
9 8 3
9 8 4
9 8 5
9 8 6
9 8 7
9 8 8
9 8 9
9 8 10
9 8 11
9 9 -11
9 9 -10
9 9 -9
9 9 -8
9 9 -7
9 9 -6
9 9 -5
9 9 -4
9 9 -3
9 9 -2
9 9 -1
9 9 0
9 9 1
9 9 2
9 9 3
9 9 4
9 9 5
9 9 6
9 9 7
9 9 8
9 9 9
9 9 10
9 9 11
9 10 -11
9 10 -10
9 10 -9
9 10 -8
9 10 -7
9 10 -6
9 10 -5
9 10 -4
9 10 -3
9 10 -2
9 10 -1
9 10 0
9 10 1
9 10 2
9 10 3
9 10 4
9 10 5
9 10 6
9 10 7
9 10 8
9 10 9
9 10 10
9 10 11
9 11 -11
9 11 -10
9 11 -9
9 11 -8
9 11 -7
9 11 -6
9 11 -5
9 11 -4
9

In [56]:
def sphere_and_cube_aoe(coord, aoe, area):
    effected_area = []
    sqrs = int(aoe/5)
    x, y, z = coord
    x_lim, y_lim, z_lim = area.shape
    low_lim = 0
    for i in range(-sqrs, sqrs):
        for j in range(-sqrs, sqrs):
            for k in range(-sqrs, sqrs):
                if i>= x_lim or j >= y_lim or k >= z_lim or i<low_lim or j<low_lim or k<low_lim:
                    pass
                else:
                    effected_area.append((i, j, k))
    return effected_area

In [57]:
ii = sphere_aoe((0,0,0), 60, x.area_map)

In [58]:
ii

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 0, 3),
 (0, 0, 4),
 (0, 0, 5),
 (0, 0, 6),
 (0, 0, 7),
 (0, 0, 8),
 (0, 0, 9),
 (0, 0, 10),
 (0, 0, 11),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 1, 5),
 (0, 1, 6),
 (0, 1, 7),
 (0, 1, 8),
 (0, 1, 9),
 (0, 1, 10),
 (0, 1, 11),
 (0, 2, 0),
 (0, 2, 1),
 (0, 2, 2),
 (0, 2, 3),
 (0, 2, 4),
 (0, 2, 5),
 (0, 2, 6),
 (0, 2, 7),
 (0, 2, 8),
 (0, 2, 9),
 (0, 2, 10),
 (0, 2, 11),
 (0, 3, 0),
 (0, 3, 1),
 (0, 3, 2),
 (0, 3, 3),
 (0, 3, 4),
 (0, 3, 5),
 (0, 3, 6),
 (0, 3, 7),
 (0, 3, 8),
 (0, 3, 9),
 (0, 3, 10),
 (0, 3, 11),
 (0, 4, 0),
 (0, 4, 1),
 (0, 4, 2),
 (0, 4, 3),
 (0, 4, 4),
 (0, 4, 5),
 (0, 4, 6),
 (0, 4, 7),
 (0, 4, 8),
 (0, 4, 9),
 (0, 4, 10),
 (0, 4, 11),
 (0, 5, 0),
 (0, 5, 1),
 (0, 5, 2),
 (0, 5, 3),
 (0, 5, 4),
 (0, 5, 5),
 (0, 5, 6),
 (0, 5, 7),
 (0, 5, 8),
 (0, 5, 9),
 (0, 5, 10),
 (0, 5, 11),
 (1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 0, 3),
 (1, 0, 4),
 (1, 0, 5),
 (1, 0, 6),
 (1, 0, 7),
 (1, 0, 8),
 (1, 0, 9),
 (1,

In [48]:
a = 12
for i in range(-a, a):
    print(i)

-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
11


In [ ]:
def cone_aoe(coord, aoe, area, direction):
    effected_area = []
    sqrs = int(aoe/5)
    x, y, z = coord
    x_lim, y_lim, z_lim = area.shape
    if direction == north:
    elif direction == north_west:
    elif direction == north_east:
    elif direction == south:
    elif direction == south_west:
    elif direction == south_east:
    elif direction == west:
    elif direction == east:
    